In [8]:
from tqdm import tqdm
import pandas as pd
import FinanceDataReader as fdr

stocks = fdr.StockListing("S&P500")
# fdr.DataReader("AAPL")

stocks_dic = {}

for i in tqdm(range(len(stocks))):   
    try:
        row = stocks.iloc[i]
        symbol = row["Symbol"]
        name = row["Name"]
        df = fdr.DataReader(symbol)    # 관련된 종목 가격데이터 
        stocks_dic[symbol] = [name, df]
    except:
        continue

100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [06:21<00:00,  1.32it/s]


In [15]:
X = []
Y = []

window = 10

for symbol in tqdm(stocks_dic):
    sample = stocks_dic[symbol][1].copy()
    name = stocks_dic[symbol][0]
    
    for i in range(len(sample) - window):
        A = sample.iloc[i : i + window].values.flatten()
        B = sample.iloc[i + window - 1, 3] * 1.05 <= sample.iloc[i + window, 3]
        X.append(A)
        Y.append(B)

 44%|███████████████████████████████████▎                                            | 221/501 [02:09<02:43,  1.71it/s]


KeyboardInterrupt: 

In [20]:
X = []
Y = []

window = 10

for symbol in tqdm(stocks_dic):
    sample = stocks_dic[symbol][1].copy().values       # 행렬로 하면 속도가 비약적으로 빨라짐
    name = stocks_dic[symbol][0]
    
    for i in range(len(sample) - window):
        A = sample[i : i + window].flatten()
        B = sample[i + window - 1, 3] * 1.05 <= sample[i + window, 3]
        X.append(A)
        Y.append(B)

100%|████████████████████████████████████████████████████████████████████████████████| 501/501 [00:12<00:00, 39.89it/s]


In [21]:
len(X), len(Y)

(4076113, 4076113)

In [22]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

train_x, test_x, train_y, test_y = train_test_split(X, Y, stratify=Y)

model = XGBClassifier()
model.fit(train_x, train_y)

pred = model.predict(test_x)

report = classification_report(test_y, pred)
print(report) 

# 정밀도 기존 0% -> 30%로까지 올림
# 목표: 80%

              precision    recall  f1-score   support

       False       0.98      1.00      0.99    997334
        True       0.31      0.02      0.04     21695

    accuracy                           0.98   1019029
   macro avg       0.65      0.51      0.51   1019029
weighted avg       0.96      0.98      0.97   1019029



In [25]:
# (시가, 저가, 고가, 종가, 거래량) X 10 : 정밀도 30%

from bs4 import BeautifulSoup
import requests
import warnings
warnings.filterwarnings("ignore")

url = requests.get("https://finance.naver.com/sise/sise_market_sum.naver")
html = BeautifulSoup(url.text)

table = html.find("table", class_ = "type_2")
table = pd.read_html(str(table))[0]
table = table.dropna(subset = "종목명")
table

,N,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,토론실
1,1.0,삼성전자,60500.0,하락 300,-0.49%,100.0,3581381.0,5919638.0,49.74,5291927.0,11.72,9.03,NaN
2,2.0,SK하이닉스,289500.0,"상승 5,500",+1.94%,5000.0,2107567.0,728002.0,55.52,1187296.0,8.11,31.06,NaN
3,3.0,삼성바이오로직스,999000.0,"상승 3,000",+0.30%,2500.0,711028.0,71174.0,12.89,7856.0,55.57,10.45,NaN
4,4.0,LG에너지솔루션,295500.0,"상승 7,500",+2.60%,500.0,691470.0,234000.0,4.09,85750.0,-59.41,-4.93,NaN
5,5.0,두산에너빌리티,69300.0,"상승 3,500",+5.32%,5000.0,443909.0,640561.0,24.43,6819475.0,-435.85,1.52,NaN
9,6.0,KB금융,112500.0,"상승 1,900",+1.72%,5000.0,429145.0,381462.0,78.22,311487.0,7.80,8.86,NaN
10,7.0,현대차,206000.0,"상승 1,000",+0.49%,5000.0,421801.0,204758.0,36.28,138124.0,4.49,12.43,NaN
11,8.0,NAVER,258500.0,"상승 1,000",+0.39%,100.0,409560.0,158437.0,46.20,559821.0,22.57,7.90,NaN
12,9.0,삼성전자우,50100.0,하락 500,-0.99%,100.0,408803.0,815975.0,73.91,414890.0,9.71,NaN,NaN
13,10.0,한화에어로스페이스,860000.0,"하락 30,000",-3.37%,5000.0,406747.0,47296.0,44.71,63725.0,17.39,53.94,NaN


## 네이버증권의 모든 표데이터와 옵션값 불러오기 가능

In [26]:
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime, timedelta
import pandas as pd
import requests
import time
import random
from numpy import nan
from io import StringIO

import os

today = str(datetime.today().date())

try:
    os.mkdir(today)
except:
    pass

def make_table(html):
    html = BeautifulSoup(url.text)
    table = html.find("table", class_ = 'type_2')
    table = pd.read_html(StringIO(str(table)))[0].dropna(subset="종목명").drop(['N', '토론실'], axis = 1)
    return table

url = requests.get("https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page=1")
html = BeautifulSoup(StringIO(url.text), 'lxml')

fields = [i['value'] for i in html.find("table", class_ = 'item_list').find_all("input", attrs = {'name' : 'fieldIds'})]

kospi_page = int(html.find("td", class_ = 'pgRR').find("a")['href'].split('=')[-1])

kospi_box = []
for n in tqdm(range(1, kospi_page + 1)):
    
    data = {'menu' : 'market_sum', 
        "fieldIds" : fields, 'returnUrl' : f"https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page={n}"}
    
    url = requests.post("https://finance.naver.com/sise/field_submit.naver", data = data)
    html = BeautifulSoup(StringIO(url.text), 'lxml')
    table = make_table(html)
    kospi_box.append(table)
    time.sleep(random.random() + 1)

kospi = pd.concat(kospi_box)
kospi['소속'] = 'KOSPI'

#--

url = requests.get("https://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page=1")
html = BeautifulSoup(url.text, 'lxml')
kosdaq_page = int(html.find("td", class_ = 'pgRR').find("a")['href'].split('=')[-1])

kosdaq_box = []
for n in tqdm(range(1, kosdaq_page + 1)):
    
    data = {'menu' : 'market_sum', 
        "fieldIds" : fields, 'returnUrl' : f"https://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page={n}"}
    
    url = requests.post("https://finance.naver.com/sise/field_submit.naver", data = data)
    html = BeautifulSoup(StringIO(url.text), 'lxml')
    table = make_table(html)
    kosdaq_box.append(table)
    time.sleep(random.random() + 1)

kosdaq = pd.concat(kosdaq_box)
kosdaq['소속'] = 'KOSDAQ'

stock = pd.concat([kospi, kosdaq], ignore_index=True)
# stock.to_excel(f"{today}_국내주식.xlsx", index=False)
stock

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [01:07<00:00,  1.87s/it]


,종목명,현재가,전일비,등락률,액면가,거래량,거래대금,전일거래량,시가,고가,...,보통주배당금,매출액증가율,영업이익증가율,외국인비율,PER,ROE,ROA,PBR,유보율,소속
0,삼성전자,60450.0,하락 350,-0.58%,100.0,5303179.0,321032.0,17340470.0,61200.0,61200.0,...,1446.0,16.20,398.34,49.74,11.71,9.03,7.10,1.02,41772.8,KOSPI
1,SK하이닉스,290500.0,"상승 6,500",+2.29%,5000.0,1253532.0,361153.0,5134863.0,288500.0,291000.0,...,2204.0,102.02,403.58,55.52,8.14,31.06,17.98,2.46,1911.2,KOSPI
2,삼성바이오로직스,996000.0,보합0,0.00%,2500.0,8354.0,8325.0,38057.0,1001000.0,1001000.0,...,0.0,23.08,18.53,12.89,55.40,10.45,6.49,6.28,6041.4,KOSPI
3,LG에너지솔루션,295500.0,"상승 7,500",+2.60%,500.0,86306.0,25439.0,219699.0,290000.0,297500.0,...,NaN,-24.08,-73.40,4.09,-59.41,-4.93,0.64,3.27,15864.8,KOSPI
4,두산에너빌리티,68900.0,"상승 3,100",+4.71%,5000.0,6874671.0,481564.0,7601019.0,70400.0,72200.0,...,NaN,-7.71,-30.65,24.43,-433.33,1.52,1.55,5.80,90.8,KOSPI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4124,아이엠,396.0,보합0,0.00%,500.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.05,-1009.39,6.22,-0.17,-133.80,-34.93,0.57,18.9,KOSDAQ
4125,더테크놀로지,380.0,보합0,0.00%,500.0,0.0,0.0,0.0,0.0,0.0,...,NaN,-30.51,31.17,1.49,-0.27,-47.83,-33.04,0.17,18.4,KOSDAQ
4126,CNH,109.0,보합0,0.00%,2500.0,0.0,0.0,0.0,0.0,0.0,...,NaN,9.69,-834.24,6.07,-0.04,-275.57,-30.95,-0.25,-112.2,KOSDAQ
4127,소프트센우,7300.0,하락 80,-1.08%,200.0,8.0,0.0,468.0,7370.0,7370.0,...,NaN,NaN,NaN,0.24,-243.33,NaN,NaN,14.47,NaN,KOSDAQ


In [27]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4129 entries, 0 to 4128
Data columns (total 33 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   종목명      4129 non-null   object 
 1   현재가      4129 non-null   float64
 2   전일비      4129 non-null   object 
 3   등락률      4129 non-null   object 
 4   액면가      4129 non-null   float64
 5   거래량      4129 non-null   float64
 6   거래대금     4129 non-null   float64
 7   전일거래량    4129 non-null   float64
 8   시가       4129 non-null   float64
 9   고가       4129 non-null   float64
 10  저가       4129 non-null   float64
 11  매수호가     4129 non-null   float64
 12  매도호가     4129 non-null   float64
 13  매수총잔량    4129 non-null   float64
 14  매도총잔량    4129 non-null   float64
 15  상장주식수    4129 non-null   float64
 16  시가총액     4129 non-null   float64
 17  매출액      2554 non-null   float64
 18  자산총계     2637 non-null   float64
 19  부채총계     2637 non-null   float64
 20  영업이익     2637 non-null   float64
 21  당기순이익    2637 

In [40]:
# PER(Price Earnings Ratio) : 주가수익비율
# 주당순이익(EPS) = 당기순이익 / 주식발행수 
# PER = 현재 주가 / 주당순이익
# 만약 PER이 10이다 -> 회사가치만큼 돈을 벌려면 10년 걸리겠구나       (PER : 작으면 작을수록 좋다)

# PBR이 1보다 작고, PER 7보다 작으면
df2 = stock[["종목명", "PER"]].dropna().sort_values("PER")
df2 = df2[(df2["PER"] > 0) & (df2["PER"] <= 7)]
df2

,종목명,PER
791,DH오토넥스,0.55
4015,씨엑스아이,0.76
3174,제이엔케이글로벌,1.00
3558,오션인더블유,1.02
1784,일정실업,1.02
...,...,...
405,LF,6.89
768,미원화학,6.90
3673,비비씨,6.91
236,현대위아,6.94


In [41]:
# ROE (Return on Equity) : 자기자본이익률(%)
# ROE = 당기순이익 / 평균자기자본 * 100              (ROE : 크면 클수록 좋다)

df2 = stock[["종목명", "ROE"]].dropna().sort_values("ROE", ascending=False)
df2
# 1억을 투자했을때 2천만원정도 이익을 냈다

,종목명,ROE
837,효성화학,10663.68
3382,E8,2823.10
1541,한창,946.65
2786,현대사료,340.50
3922,선샤인푸드,286.29
...,...,...
3550,푸드나무,-364.32
3965,세종메디칼,-375.00
2721,푸른소나무,-417.81
3290,제일엠앤에스,-791.40


In [45]:
# PBR (Price to Book Ratio) : 주가순자산비율
# PBR 보통 1이어야 정상
# PBR > 1 원래 받아야하는 평가보다 더 높게 받고 있다
# PBR < 1 회사가 좀더 저렴하게 평가받고있다... 이런경우 추후에 1로 오를 것

df2 = stock[["종목명", "PBR"]].dropna()
df2 = df2[df2["PBR"] < 0.7]
df2
# 이런종목에 투자하기 좋다

,종목명,PBR
6,현대차,0.49
10,기아,0.69
13,신한지주,0.53
16,현대모비스,0.55
20,한국전력,0.60
...,...,...
4108,올리패스,0.65
4121,투비소프트,0.32
4124,아이엠,0.57
4125,더테크놀로지,0.17


In [48]:
# 고ROE + 저PBR 가성비 좋은 우량주
# ROE가 높으면서 PBR낮으면 저평가받고있으면서 수익률은 높다 = 투자가치 있다고 볼수있음
# PBR < 1

df2 = stock[["종목명", "ROE", 'PBR', "PER"]].dropna()
df2[(df2["ROE"] >= 15) & (df2["PBR"] < 1) & (df2["PER"] < 7)]

,종목명,ROE,PBR,PER
10,기아,19.09,0.69,4.17
23,HMM,15.35,0.72,4.47
62,DB손해보험,18.98,0.91,5.08
152,현대해상,15.75,0.52,3.50
205,에스엘,17.42,0.62,4.17
...,...,...,...,...
3674,백금T&A,19.95,0.64,3.50
3727,에코캡,20.43,0.38,2.15
3871,기산텔레콤,16.39,0.66,3.87
3922,선샤인푸드,286.29,-3.98,-3.19
